In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
n = 5
data = pd.DataFrame(columns=['topic', 'text'])
for i in range(1, n+1):
  new_topic = pd.read_csv(f"./drive/MyDrive/requests_gpt_{i}.csv", header=None)
  new_topic.columns = ['topic', 'text']
  data = pd.concat([data, new_topic])
data.drop(data[data['topic'] == 0].index, inplace = True)
print(data['topic'].value_counts())
data

topic
1    155
5    152
4    150
3    140
2    140
Name: count, dtype: int64


,topic,text
0,1,Каким образом можно предъявить иск о задолженн...
1,1,Каковы основания для обращения с иском о задол...
2,1,Какие органы могут принимать и рассматривать и...
3,1,Какие последствия могут возникнуть в случае не...
4,1,Каково количество и размер просроченных платеж...
...,...,...
147,5,Какие последующие шаги следует предпринять пос...
148,5,Как учитывать прецеденты и практику судебных р...
149,5,Поставщик взял деньги и не поставил товар
150,5,Не поставил товар


In [4]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
X = tokenizer(list(data['text']), padding=True, truncation=True, return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
data['topic'] = data['topic'] - 1

In [6]:
# Преобразование меток в категории
y = tf.keras.utils.to_categorical(data['topic'], num_classes=5)

# Преобразование тензоров в массивы NumPy
input_ids = X['input_ids'].numpy()
attention_mask = X['attention_mask'].numpy()

In [7]:
# Разделение данных на обучающую и проверочную выборки
X_train_ids, X_val_ids, y_train, y_val = train_test_split(input_ids, y, test_size=0.2, random_state=42)
X_train_mask, X_val_mask = train_test_split(attention_mask, test_size=0.2, random_state=42)

In [8]:
# Определение модели
max_length = 31
input_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# Загрузка модели с PyTorch весами
bert_model = TFBertModel.from_pretrained('DeepPavlov/rubert-base-cased', from_pt=True)
bert_outputs = bert_model(input_ids_layer, attention_mask=attention_mask_layer)
pooled_output = bert_outputs.pooler_output

dense_layer = tf.keras.layers.Dense(512, activation='relu')(pooled_output)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense_layer)
output_layer = tf.keras.layers.Dense(5, activation='softmax')(dropout_layer)

model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output_layer)

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Вывод структуры модели
model.summary()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 31)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 31)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1778534   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 31, 76                                       

In [9]:
# Обучение модели
model.fit(
    {'input_ids': X_train_ids, 'attention_mask': X_train_mask},
    y_train,
    epochs=5,
    batch_size=8,
    validation_data=({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)
)

Epoch 1/5
74/74 [==============================] - 117s 316ms/step - loss: 1.3779 - accuracy: 0.3956 - val_loss: 0.2397 - val_accuracy: 0.9865
Epoch 2/5
74/74 [==============================] - 10s 136ms/step - loss: 0.1378 - accuracy: 0.9762 - val_loss: 0.0448 - val_accuracy: 0.9932
Epoch 3/5
74/74 [==============================] - 10s 138ms/step - loss: 0.0197 - accuracy: 1.0000 - val_loss: 0.0316 - val_accuracy: 0.9932
Epoch 4/5
74/74 [==============================] - 10s 131ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0337 - val_accuracy: 0.9932
Epoch 5/5
74/74 [==============================] - 9s 123ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0364 - val_accuracy: 0.9865


In [10]:
# Пример нового текста для предсказания
new_texts = ["Меня обокрали. Что делать?"]
new_X = tokenizer(new_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

# Преобразование новых тензоров в массивы NumPy
new_input_ids = new_X['input_ids'].numpy()
new_attention_mask = new_X['attention_mask'].numpy()

# Предсказание класса с использованием обученной модели
predictions = model.predict({'input_ids': new_input_ids, 'attention_mask': new_attention_mask})

# Конвертация вероятностей в предсказанные классы
predicted_classes = np.argmax(predictions, axis=1)

print("Предсказанные классы:", predicted_classes)

1/1 [==============================] - 13s 13s/step
Предсказанные классы: [3]


In [16]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
predictions = model.predict({'input_ids': X_val_ids, 'attention_mask': X_val_mask})
predicted_classes = np.argmax(predictions, axis=1)
y_val = np.argmax(y_val, axis=1)
print('accuracy',accuracy_score(y_val, predicted_classes))
print('precision',precision_score(y_val, predicted_classes, average='macro'))
print('recall',recall_score(y_val, predicted_classes, average='macro'))

5/5 [==============================] - 1s 105ms/step
accuracy 0.9864864864864865
precision 0.9854219948849104
recall 0.9854219948849104


In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_val, predicted_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        27
           2       0.97      0.97      0.97        34
           3       0.96      0.96      0.96        23
           4       1.00      1.00      1.00        30

    accuracy                           0.99       148
   macro avg       0.99      0.99      0.99       148
weighted avg       0.99      0.99      0.99       148

